In [1]:
import pandas as pd
import numpy as np
import json
from pyearth import Earth
import plotly.graph_objects as go
import datetime as dt
from re import findall
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
import copy

In [2]:
def adjust_dt(df, delta_time):
    date_init =  df['snapped_at'][0]
    idx_date = df['snapped_at'].map(dt.datetime.toordinal)
    idx_date = idx_date - idx_date[0]
    df_modify = df[df['snapped_at'] == date_init]
    for i in idx_date:
        if i!=0:
            if i%delta_time == 0:
                df_modify = pd.concat([df_modify,df[df['snapped_at'] == date_init+pd.Timedelta(i,'d')]])
    return df_modify

In [3]:
f = open('df_checked_dict.json')
data = json.load(f)

In [4]:
data_pump = pd.read_csv('data.csv').drop(['Unnamed: 0','Unnamed: 0.1'], axis=1)
data_pump['base_date'] = pd.to_datetime(data_pump['base_date'], format='%d-%m-%Y')
data_pump['max_date'] = pd.to_datetime(data_pump['max_date'], format='%d-%m-%Y')
data_pump['pump_length'] = data_pump['max_date'] - data_pump['base_date']

In [84]:
data_pump.iloc[374]

coin                              shib
base_date          2021-10-04 00:00:00
base_price                 6.05672e-06
max_date           2021-10-28 00:00:00
max_price                  8.19045e-05
pct_change                     12.5229
pump_length           24 days 00:00:00
date_listing                01-08-2020
pump_to_listing                    429
state                          bullish
Name: 374, dtype: object

In [5]:
PCT_THRESHOLD = 0.2
REGION_THRESHOLD = 30
FAKE_PUMP_THRESHOLD = 4
FAKE_PUMP_BOUNDARY = 1
arr_is_matched = [-1 for i in range(len(data_pump))]
list_staircase = []
for p in range(len(data_pump)):
# for p in [374]:
    print(p)
    coin = data_pump['coin'][p]
    temp_df = pd.DataFrame.from_dict(data[coin]).dropna()
    temp_df['snapped_at'] = pd.to_datetime(temp_df['snapped_at'], format='%Y-%m-%d %H:%M:%S UTC')
    temp_df = temp_df[temp_df['snapped_at'] <= data_pump['base_date'][p]]
    dict_staircase = {}
    for t in range(7):
        temp_df2 = adjust_dt(temp_df,7-t)
        temp_df2_up = temp_df2[np.abs(temp_df2['pct_change']) > PCT_THRESHOLD]
        temp_df2_up['region_length'] = temp_df2_up['snapped_at'].diff()
        temp_df2_region = temp_df2_up[temp_df2_up['region_length'] >= pd.Timedelta(REGION_THRESHOLD,'d')]
        is_staircase = False
        staircase = {'date_before': [],
                    'date_after': []}
        for j in range(len(temp_df2_region)):
            date_init = temp_df2_region['snapped_at'][j] - temp_df2_region['region_length'][j]
            date_final = temp_df2_region['snapped_at'][j]
            temp_df2_filter = temp_df2[np.logical_and((temp_df2['snapped_at'] >= date_init),(temp_df2['snapped_at'] < date_final))]
            x = temp_df2_filter['snapped_at'].map(dt.datetime.toordinal)
            x = x - x[0]
            scaler = MinMaxScaler()
            x = scaler.fit_transform(x.values.reshape(-1,1))
            y = temp_df2_filter['price']
            model = Earth()
            model.fit(x,y)
            y_pred = model.predict(x)

            sum_li = model.summary().split('\n')[4:-2]
            arr_knot = []
            for i in range(len(sum_li)):
                line = sum_li[i]
                if 'Yes' not in line:
                    if 'x0' in line:
                        columns = line.split()
                        try:
                            f = findall(r'\((.+)\)',columns[0])[0]
                            const = findall(r'[+-]?[0-9]\.[0-9]+',f)
                            const = float(const[0])
                            knot = abs(scaler.inverse_transform([[const]])[0][0])
                            knot = round(knot/(7-t))*(7-t)
                            if knot < len(x):
                                arr_knot.append(knot)
                        except Exception as e: 
                            continue
            arr_knot = np.unique(arr_knot)
            # print(sum_li)
            arr_grad = []
            arr_date = [date_init]
            for i in range(len(arr_knot)+1):
                if i == 0:
                    x0 = 0
                else:
                    x0 = int(arr_knot[i-1])

                if i == len(arr_knot):
                    x1 = (len(y_pred)-1)*(7-t)
                else:
                    x1 = int(arr_knot[i])
                y0 = y_pred[int(x0/(7-t))]
                y1 = y_pred[int(x1/(7-t))]
                grad = (y1-y0)/(x1-x0)
                arr_grad.append(grad)
                arr_date.append(date_init + pd.Timedelta(x1,'d'))
            for i in range(len(arr_grad)-1):
                grad_before = arr_grad[i]
                grad_after = arr_grad[i+1]
                date_before = arr_date[i]
                date_after = arr_date[i+2]
                delta_grad = (abs(grad_after)-abs(grad_before))
                if grad_before>0 and grad_before*grad_after < 0 and delta_grad/abs(grad_before) < 0.5:
                    is_staircase = True
                    staircase['date_before'].append(date_before)
                    staircase['date_after'].append(date_after)
        staircase['fake_pump'] = [0 for i in range(len(staircase['date_before']))]
        dict_staircase[t] = staircase
        last_day = temp_df2['snapped_at'][-1]
        is_fake_pump = False
        for i in range(len(staircase['date_before'])):
            date_after = staircase['date_after'][i]
            is_search = True
            date_now = date_after
            data_avg = temp_df2[temp_df2['snapped_at'] <= date_after]['pct_change'].values[-2:].tolist()
            data_avg = [abs(d) for d in data_avg]
            is_fake_pump = False
            while is_search:
                temp_df3 = temp_df2[temp_df2['snapped_at'] >= date_now]
                temp_df3 = temp_df3[temp_df3['snapped_at'] <= date_now+pd.Timedelta(7-t,'d')]
                try:
                    p0 = temp_df3['pct_change'].values[0]
                    p1 = temp_df3['pct_change'].values[1]
                except:
                    date_now = date_now+pd.Timedelta(7-t,'d')
                    if i != len(staircase['date_before'])-1:
                        if date_now >= staircase['date_before'][i+1]:
                            is_search = False
                    if date_now >= last_day:
                        is_search = False
                    continue
                avg = np.average(data_avg)
                if p0*p1 <= 0:
                    if abs(p0) > FAKE_PUMP_THRESHOLD*avg and abs(p1) > FAKE_PUMP_THRESHOLD*avg and abs(p0) < FAKE_PUMP_BOUNDARY and abs(p1) < FAKE_PUMP_BOUNDARY:
                        is_fake_pump = True
                        is_search = False
                        staircase['fake_pump'][i] = temp_df3['snapped_at'][0]
                        print('found')
                if not is_fake_pump:
                    data_avg.append(abs(p0))
                date_now = date_now+pd.Timedelta(7-t,'d')
                if i != len(staircase['date_before'])-1:
                    if date_now >= staircase['date_before'][i+1]:
                        is_search = False
                if date_now == last_day:
                    is_search = False
                if abs(p0) > 0.5:
                    is_search = False
        if is_fake_pump:
            arr_is_matched[p] = 7-t
            break
    list_staircase.append(dict_staircase)
data_pump['is_match'] = arr_is_matched

0
1
2
3
4
found
5
6
found
7
8
9
10
found
11
12
13
found
14
found
found
15
found
16
found
17
found
18
found
19
found
20
found
21
found
found
found
22
found
found
found
23
found
found
found
24
found
found
found
25
26
found
found
27
found
found
28
29
30
31
32
33
found
34
35
found
found
found
36
found
37
found
38
39
found
40
found
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
found
57
58
found
59
60
found
found
61
found
found
62
found
found
63
found
found
found
found
found
64
65
66
67
68
found
69
found
found
70
found
71
72
found
73
74
found
75
found
76
77
78
79
80
81
82
found
83
found
84
found
85
86
87
88
89
found
90
found
found
91
found
found
92
93
found
94
found
95
found
96
97
found
98
99
100
101
102
103
104
105
106
107
found
found
108
109
found
110
111
112
113
114
115
116
found
117
found
118
found
119
120
121
122
found
123
found
124
found
125
found
126
found
found
127
128
129
130
131
132
133
134
135
136
found
137
found
138
found
found
found
found
139
140
141
found
142
143
found
found


In [161]:
len(data_pump[data_pump['is_match']!=-1])

114

In [158]:
len(data_pump)

481

In [142]:
temp_df2[temp_df2['snapped_at'] >= date_now]

,snapped_at,price,market_cap,total_volume,pct_change
111,2018-02-06,2.045406,1.227243e+07,514947.615462,-0.223304
115,2018-02-10,2.490431,1.494259e+07,166712.049327,0.004258
117,2018-02-12,2.178037,1.306822e+07,275814.638446,-0.088992
119,2018-02-14,2.179457,1.307674e+07,91068.365743,-0.040957
121,2018-02-16,2.353425,1.412055e+07,197208.007698,-0.003585
...,...,...,...,...,...
1177,2021-01-07,0.158949,1.360994e+06,74330.754599,-0.000521
1179,2021-01-09,0.125163,1.038179e+06,88063.497511,-0.117410
1181,2021-01-11,0.102456,8.332793e+05,59936.298574,-0.257714
1183,2021-01-13,0.112069,9.036027e+05,64029.959148,0.195577


In [130]:
staircase

{'date_before': [Timestamp('2020-06-23 00:00:00')],
 'date_after': [Timestamp('2020-08-03 00:00:00')],
 'fake_pump': [0]}

Timestamp('2020-08-06 00:00:00')

# Forecast

In [162]:
f = open('df_checked_dict.json')
data = json.load(f)

df_checked = {}
for i in data.keys():
    df_checked[i] = pd.DataFrame.from_dict(data[i])
    df_checked[i]['snapped_at'] = pd.to_datetime(df_checked[i]['snapped_at'], format='%Y-%m-%d %H:%M:%S UTC')

In [179]:
dict_forecast = {'coin': [],
                'timeframe': [],
                'detail': []}
for c in data.keys():
    temp_df = copy.deepcopy(df_checked[c])
    c_pump = data_pump[data_pump['coin'] == c]
    if len(c_pump) >= 1:
        temp_df = temp_df[temp_df['snapped_at'] >= max(c_pump['base_date'])]
    dict_staircase = {}
    for t in range(7):
        temp_df2 = adjust_dt(temp_df,7-t)
        temp_df2_up = temp_df2[np.abs(temp_df2['pct_change']) > PCT_THRESHOLD]
        temp_df2_up['region_length'] = temp_df2_up['snapped_at'].diff()
        temp_df2_region = temp_df2_up[temp_df2_up['region_length'] >= pd.Timedelta(REGION_THRESHOLD,'d')]
        is_staircase = False
        staircase = {'date_before': [],
                    'date_after': []}
        for j in range(len(temp_df2_region)):
            date_init = temp_df2_region['snapped_at'][j] - temp_df2_region['region_length'][j]
            date_final = temp_df2_region['snapped_at'][j]
            temp_df2_filter = temp_df2[np.logical_and((temp_df2['snapped_at'] >= date_init),(temp_df2['snapped_at'] < date_final))]
            x = temp_df2_filter['snapped_at'].map(dt.datetime.toordinal)
            x = x - x[0]
            scaler = MinMaxScaler()
            x = scaler.fit_transform(x.values.reshape(-1,1))
            y = temp_df2_filter['price']
            model = Earth()
            model.fit(x,y)
            y_pred = model.predict(x)

            sum_li = model.summary().split('\n')[4:-2]
            arr_knot = []
            for i in range(len(sum_li)):
                line = sum_li[i]
                if 'Yes' not in line:
                    if 'x0' in line:
                        columns = line.split()
                        try:
                            f = findall(r'\((.+)\)',columns[0])[0]
                            const = findall(r'[+-]?[0-9]\.[0-9]+',f)
                            const = float(const[0])
                            knot = abs(scaler.inverse_transform([[const]])[0][0])
                            knot = round(knot/(7-t))*(7-t)
                            if knot < len(x):
                                arr_knot.append(knot)
                        except Exception as e: 
                            continue
            arr_knot = np.unique(arr_knot)
            # print(sum_li)
            arr_grad = []
            arr_date = [date_init]
            for i in range(len(arr_knot)+1):
                if i == 0:
                    x0 = 0
                else:
                    x0 = int(arr_knot[i-1])

                if i == len(arr_knot):
                    x1 = (len(y_pred)-1)*(7-t)
                else:
                    x1 = int(arr_knot[i])
                y0 = y_pred[int(x0/(7-t))]
                y1 = y_pred[int(x1/(7-t))]
                grad = (y1-y0)/(x1-x0)
                arr_grad.append(grad)
                arr_date.append(date_init + pd.Timedelta(x1,'d'))
            for i in range(len(arr_grad)-1):
                grad_before = arr_grad[i]
                grad_after = arr_grad[i+1]
                date_before = arr_date[i]
                date_after = arr_date[i+2]
                delta_grad = (abs(grad_after)-abs(grad_before))
                if grad_before>0 and grad_before*grad_after < 0 and delta_grad/abs(grad_before) < 0.5:
                    is_staircase = True
                    staircase['date_before'].append(date_before)
                    staircase['date_after'].append(date_after)
        staircase['fake_pump'] = [pd.Timedelta(0,'d') for i in range(len(staircase['date_before']))]
        dict_staircase[t] = staircase
        last_day = temp_df2['snapped_at'][-1]
        is_fake_pump = False
        for i in range(len(staircase['date_before'])):
            date_after = staircase['date_after'][i]
            is_search = True
            date_now = date_after
            data_avg = temp_df2[temp_df2['snapped_at'] <= date_after]['pct_change'].values[-2:].tolist()
            data_avg = [abs(d) for d in data_avg]
            is_fake_pump = False
            while is_search:
                temp_df3 = temp_df2[temp_df2['snapped_at'] >= date_now]
                temp_df3 = temp_df3[temp_df3['snapped_at'] <= date_now+pd.Timedelta(7-t,'d')]
                try:
                    p0 = temp_df3['pct_change'].values[0]
                    p1 = temp_df3['pct_change'].values[1]
                except:
                    date_now = date_now+pd.Timedelta(7-t,'d')
                    if i != len(staircase['date_before'])-1:
                        if date_now >= staircase['date_before'][i+1]:
                            is_search = False
                    if date_now >= last_day:
                        is_search = False
                    continue
                avg = np.average(data_avg)
                if p0*p1 <= 0:
                    if abs(p0) > FAKE_PUMP_THRESHOLD*avg and abs(p1) > FAKE_PUMP_THRESHOLD*avg and abs(p0) < FAKE_PUMP_BOUNDARY and abs(p1) < FAKE_PUMP_BOUNDARY:
                        is_fake_pump = True
                        is_search = False
                        staircase['fake_pump'][i] = temp_df3['snapped_at'][0]
                        # print('found')
                if not is_fake_pump:
                    data_avg.append(abs(p0))
                date_now = date_now+pd.Timedelta(7-t,'d')
                if i != len(staircase['date_before'])-1:
                    if date_now >= staircase['date_before'][i+1]:
                        is_search = False
                if date_now == last_day:
                    is_search = False
                if abs(p0) > 0.5:
                    is_search = False
        if is_fake_pump:
            idx_last_staircase = np.where(np.array(staircase['fake_pump']) != pd.Timedelta(0,'d'))[0][-1]
            print(c, 7-t, staircase['date_before'][idx_last_staircase], staircase['date_after'][idx_last_staircase], staircase['fake_pump'][idx_last_staircase])
            dict_forecast['coin'].append(c)
            dict_forecast['timeframe'].append(7-t)
            dict_forecast['detail'].append({'date_before': staircase['date_before'][idx_last_staircase],
                                            'date_after' : staircase['date_after'][idx_last_staircase],
                                            'fake_pump' : staircase['fake_pump'][idx_last_staircase]})

    

adp 1 2021-11-03 00:00:00 2022-01-02 00:00:00 2022-01-11 00:00:00
adx 3 2018-12-01 00:00:00 2019-10-03 00:00:00 2019-10-06 00:00:00
ae 3 2019-03-27 00:00:00 2019-04-26 00:00:00 2019-06-25 00:00:00
aion 5 2018-03-27 00:00:00 2019-08-19 00:00:00 2021-02-19 00:00:00
amp 1 2022-01-27 00:00:00 2022-03-13 00:00:00 2022-05-12 00:00:00
anc 3 2021-04-19 00:00:00 2021-06-18 00:00:00 2021-06-21 00:00:00
apx 1 2022-03-03 00:00:00 2022-04-04 00:00:00 2022-04-05 00:00:00
ardr 3 2018-03-31 00:00:00 2018-11-14 00:00:00 2021-09-05 00:00:00
aria20 2 2021-07-17 00:00:00 2022-01-07 00:00:00 2022-02-08 00:00:00
asd 1 2021-10-02 00:00:00 2021-11-03 00:00:00 2021-11-04 00:00:00
ast 2 2022-01-26 00:00:00 2022-03-07 00:00:00 2022-05-12 00:00:00
auction 1 2022-01-26 00:00:00 2022-03-19 00:00:00 2022-03-20 00:00:00
auto 1 2021-10-30 00:00:00 2021-12-17 00:00:00 2021-12-29 00:00:00
bake 1 2022-01-28 00:00:00 2022-02-27 00:00:00 2022-04-01 00:00:00
band 2 2021-09-27 00:00:00 2022-06-08 00:00:00 2022-06-10 00:00:00

In [185]:
arr_detail = []
for i in range(len(dict_forecast['detail'])):
    arr_detail.append({'date_before': str(dict_forecast['detail'][i]['date_before']),
                        'date_after': str(dict_forecast['detail'][i]['date_after']),
                        'fake_pump': str(dict_forecast['detail'][i]['fake_pump'])})

In [187]:
dict_forecast2 = copy.deepcopy(dict_forecast)
dict_forecast2['detail'] = arr_detail
with open("forecast.json", "w") as outfile:
    json.dump(dict_forecast2, outfile)

In [31]:
data_pump = pd.read_csv('small_pump.csv').drop(['Unnamed: 0'], axis=1)
data_pump['base_date'] = pd.to_datetime(data_pump['base_date'], format='%Y-%m-%d')
data_pump['max_date'] = pd.to_datetime(data_pump['max_date'], format='%Y-%m-%d')
data_pump['pump_length'] = data_pump['max_date'] - data_pump['base_date']

PCT_THRESHOLD = 0.25
# base_date_new = df_data_pump['base_date']
for i in data_pump.index:
    coin = data_pump.iloc[i]['coin']
    base_date = data_pump.iloc[i]['base_date']
    temp_df = pd.DataFrame.from_dict(data[coin]).dropna()
    temp_df['snapped_at'] = pd.to_datetime(temp_df['snapped_at'], format='%Y-%m-%d %H:%M:%S UTC')
    temp_df['ma'] = temp_df['price'].rolling(window=7).mean()
    temp_df = temp_df.dropna()
    temp_df = temp_df[temp_df['snapped_at'] >= base_date]
    temp_df['pct_to_ma'] = (temp_df['price'] - temp_df['ma'])/temp_df['ma']
    temp_df = temp_df[temp_df['pct_to_ma'] >= PCT_THRESHOLD]
    try:
        data_pump['base_date'][i] = temp_df['snapped_at'][0] - pd.Timedelta(1,'d')
    except Exception as e: print(coin, base_date)

10set 2022-02-02 00:00:00
10set 2022-06-09 00:00:00
ads 2022-03-08 00:00:00
ae 2021-09-30 00:00:00
aethc 2021-03-25 00:00:00
aethc 2021-07-21 00:00:00
alpaca 2022-02-25 00:00:00
atom 2021-12-18 00:00:00
ava 2021-07-21 00:00:00
avax 2021-10-13 00:00:00
axel 2020-12-11 00:00:00
bal 2021-07-21 00:00:00
band 2021-07-21 00:00:00
bit 2021-09-22 00:00:00
btc 2020-10-19 00:00:00
btse 2021-02-24 00:00:00
cake 2021-07-21 00:00:00
ccd 2022-03-25 00:00:00
cet 2021-10-01 00:00:00
ceth 2021-03-25 00:00:00
ceth 2021-07-21 00:00:00
chsb 2021-07-21 00:00:00
cuni 2021-07-21 00:00:00
czrx 2021-07-21 00:00:00
czrx 2022-03-08 00:00:00
dao 2021-10-31 00:00:00
dash 2021-03-26 00:00:00
dash 2021-07-17 00:00:00
dot 2021-07-21 00:00:00
elg 2021-02-04 00:00:00
eth 2021-03-25 00:00:00
eth 2021-07-21 00:00:00
firo 2021-07-21 00:00:00
grt 2021-07-21 00:00:00
gt 2021-07-21 00:00:00
gt 2021-09-29 00:00:00
ht 2021-07-21 00:00:00
hxro 2021-03-26 00:00:00
imx 2022-03-07 00:00:00
kava 2021-06-22 00:00:00
kcs 2021-09-22 0

In [5]:
f = open('forecast.json')
data_forecast = json.load(f)

In [32]:
data_forecast_rev = {'coin': [],
                'timeframe': [],
                'detail': []}
for i in range(len(data_forecast['coin'])):
    coin = data_forecast['coin'][i]
    dt = data_forecast['timeframe'][i]
    detail = data_forecast['detail'][i]
    date_fp = pd.to_datetime(detail['fake_pump'], format='%Y-%m-%d %H:%M:%S')
    temp_df = data_pump[data_pump['coin'] == coin]
    try:
        if date_fp > max(temp_df['base_date']):
            data_forecast_rev['coin'].append(coin)
            data_forecast_rev['timeframe'].append(dt)
            data_forecast_rev['detail'].append(detail)
    except ValueError:
        data_forecast_rev['coin'].append(coin)
        data_forecast_rev['timeframe'].append(dt)
        data_forecast_rev['detail'].append(detail)
    

In [43]:
data_forecast_rev2 = {'coin': [],
                'timeframe': [],
                'detail': []}
for i in range(len(data_forecast_rev['coin'])):
    coin = data_forecast_rev['coin'][i]
    dt = data_forecast_rev['timeframe'][i]
    detail = data_forecast_rev['detail'][i]
    date_fp = pd.to_datetime(detail['date_before'], format='%Y-%m-%d %H:%M:%S')
    temp_df = data_pump[data_pump['coin'] == coin]
    try:
        if date_fp > pd.to_datetime('01-01-2022', format='%d-%m-%Y'):
            data_forecast_rev2['coin'].append(coin)
            data_forecast_rev2['timeframe'].append(dt)
            data_forecast_rev2['detail'].append(detail)
    except ValueError:
        data_forecast_rev2['coin'].append(coin)
        data_forecast_rev2['timeframe'].append(dt)
        data_forecast_rev2['detail'].append(detail)

In [44]:
len(data_forecast_rev2['coin'])

74

In [48]:
idx = 2
print(data_forecast_rev2['coin'][idx])
print(data_forecast_rev2['timeframe'][idx])
print(data_forecast_rev2['detail'][idx])

auction
1
{'date_before': '2022-01-26 00:00:00', 'date_after': '2022-03-19 00:00:00', 'fake_pump': '2022-03-20 00:00:00'}


In [55]:
for i in range(len(data_forecast_rev2['coin'])):
    # if data_forecast_rev2['timeframe'][i] == 1 or data_forecast_rev2['timeframe'][i] == 7:
    print(data_forecast_rev2['coin'][i], data_forecast_rev2['timeframe'][i], data_forecast_rev2['detail'][i]['date_before'], data_forecast_rev2['detail'][i]['date_after'], data_forecast_rev2['detail'][i]['fake_pump'])

amp 1 2022-01-27 00:00:00 2022-03-13 00:00:00 2022-05-12 00:00:00
apx 1 2022-03-03 00:00:00 2022-04-04 00:00:00 2022-04-05 00:00:00
auction 1 2022-01-26 00:00:00 2022-03-19 00:00:00 2022-03-20 00:00:00
bake 1 2022-01-28 00:00:00 2022-02-27 00:00:00 2022-04-01 00:00:00
bfc 1 2022-01-27 00:00:00 2022-02-21 00:00:00 2022-05-10 00:00:00
bmx 1 2022-03-16 00:00:00 2022-04-10 00:00:00 2022-05-13 00:00:00
bor 1 2022-03-11 00:00:00 2022-04-11 00:00:00 2022-04-12 00:00:00
btm 1 2022-03-10 00:00:00 2022-04-24 00:00:00 2022-04-27 00:00:00
btr 2 2022-01-02 00:00:00 2022-02-25 00:00:00 2022-05-12 00:00:00
cream 2 2022-03-10 00:00:00 2022-05-11 00:00:00 2022-05-13 00:00:00
crts 1 2022-04-02 00:00:00 2022-04-22 00:00:00 2022-06-18 00:00:00
ctxc 1 2022-01-27 00:00:00 2022-03-08 00:00:00 2022-04-05 00:00:00
cvxcrv 1 2022-03-13 00:00:00 2022-05-04 00:00:00 2022-05-05 00:00:00
cxo 1 2022-01-25 00:00:00 2022-02-24 00:00:00 2022-04-29 00:00:00
dag 1 2022-03-10 00:00:00 2022-04-09 00:00:00 2022-04-12 00:00:0

In [54]:
print(data_forecast_rev2)

{'coin': ['amp', 'apx', 'auction', 'bake', 'bfc', 'bmx', 'bor', 'btm', 'btr', 'cream', 'crts', 'ctxc', 'cvxcrv', 'cxo', 'dag', 'dero', 'dnt', 'dome', 'dxd', 'efi', 'elon', 'emaid', 'fio', 'fox', 'fsn', 'go', 'gst-sol', 'gxc', 'hns', 'hoo', 'kishu', 'klv', 'kmd', 'koda', 'krl', 'kub', 'lend', 'lrc', 'lsk', 'mmf', 'mxc', 'mxc', 'myst', 'nmr', 'nrv', 'ooki', 'oxy', 'peak', 'plu', 'pro', 'quack', 'raca', 'rad', 'sb', 'scp', 'snow', 'solve', 'sprt', 'starl', 'sure', 'tomb', 'ton', 'usv', 'velo', 'vgx', 'vro', 'wicc', 'wmt', 'xdata', 'xmr', 'xvs', 'xyo', 'yld', 'zrx'], 'timeframe': [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'detail': [{'date_before': '2022-01-27 00:00:00', 'date_after': '2022-03-13 00:00:00', 'fake_pump': '2022-05-12 00:00:00'}, {'date_before': '2022-03-03 00:00:00', 'date_after': '2022-04-04 00:0